### **<h2>Обработка ошибок в REST API</h2>**

#### **Типовые ошибки и коды состояния**

REST API использует коды состояния HTTP, чтобы информировать клиентов о результате выполнения запроса. Для правильной работы важно использовать эти коды согласно стандарту, что помогает клиентам API интерпретировать результаты запросов. DRF имеет встроенные механизмы для обработки ошибок, которые также можно кастомизировать.

**Основные категории ошибок**:

1. **Ошибки клиента (4xx)** — возникают при проблемах на стороне клиента:

   - **<span style="color: #85004B;">400 Bad Request</span>** — ошибка, связанная с неправильным синтаксисом или недостающими данными в запросе.
   - **<span style="color: #85004B;">401 Unauthorized</span>** — клиент не аутентифицирован, а доступ к ресурсу ограничен.
   - **<span style="color: #85004B;">403 Forbidden</span>** — клиент аутентифицирован, но не имеет прав для доступа к ресурсу.
   - **<span style="color: #85004B;">404 Not Found</span>** — запрашиваемый ресурс не найден.
   - **<span style="color: #85004B;">405 Method Not Allowed</span>** — HTTP-метод, указанный в запросе, не поддерживается для данного ресурса.
   - **<span style="color: #85004B;">429 Too Many Requests</span>** — клиент превысил лимит запросов за определенный период.

2. **Ошибки сервера (5xx)** — указывают на проблемы на стороне сервера:
   - **<span style="color: #85004B;">500 Internal Server Error</span>** — общая ошибка сервера.
   - **<span style="color: #85004B;">502 Bad Gateway</span>** — сервер получил некорректный ответ от другого сервера.
   - **<span style="color: #85004B;">503 Service Unavailable</span>** — сервер временно недоступен.
   - **<span style="color: #85004B;">504 Gateway Timeout</span>** — сервер не получил своевременного ответа от другого сервера.


#### **Форматирование ошибок в JSON**

Для удобства клиентов, ошибки в ответах API следует стандартизировать. DRF позволяет настраивать JSON-ответы на ошибки, делая их более детализированными.

**Структура JSON-ответов об ошибках**:

Обычно JSON-ответ на ошибку содержит такие ключи, как `error` (тип ошибки), `status` (HTTP-код состояния) и `details` (подробности):

```json
{
    "error": "Invalid request",
    "status": 400,
    "details": "The 'username' field is required."
}
```


### **Пример настройки кастомного обработчика ошибок в Django REST Framework**

Для настройки единого формата ответов на ошибки создадим кастомный обработчик исключений, который будет возвращать JSON в стандартизированном виде.

1. **Создание обработчика ошибок**

   Сначала создадим файл `exceptions.py` для обработки ошибок в корневой директории приложения.

   ```plaintext
   my_project/
   ├── my_app/
   │   ├── views.py
   │   ├── models.py
   │   └── exceptions.py  # новый файл
   ```

   В `exceptions.py` добавим функцию `custom_exception_handler`, которая будет переопределять стандартное поведение DRF:

   ```python
   # exceptions.py
   from rest_framework.views import exception_handler
   from rest_framework.response import Response
   from rest_framework import status

   def custom_exception_handler(exc, context):
       # Вызов стандартного обработчика DRF для получения стандартного ответа
       response = exception_handler(exc, context)
       
       # Если DRF смог обработать исключение
       if response is not None:
           custom_response_data = {
               'error': 'Invalid request',
               'status': response.status_code,
               'details': response.data  # данные об ошибке от DRF
           }
           return Response(custom_response_data, status=response.status_code)
       
       # Для непредвиденных ошибок (например, 500 Internal Server Error)
       return Response(
           {'error': 'Internal server error', 'status': status.HTTP_500_INTERNAL_SERVER_ERROR},
           status=status.HTTP_500_INTERNAL_SERVER_ERROR
       )
   ```

2. **Подключение кастомного обработчика в настройках**:

   В `settings.py` укажем путь к нашему обработчику исключений:

   ```python
   # settings.py
   REST_FRAMEWORK = {
       'EXCEPTION_HANDLER': 'my_app.exceptions.custom_exception_handler',
   }
   ```

   После этого все ошибки, возникающие в API, будут обрабатываться через `custom_exception_handler`, что обеспечивает стандартизированный формат JSON-ответов.


### **Кастомизация отдельных ошибок в представлениях**

В некоторых случаях полезно обрабатывать ошибки на уровне конкретных представлений, чтобы возвращать детализированные сообщения. Например, если данные, отправленные клиентом, не соответствуют требованиям, мы можем явно указать недостающие или некорректные поля.

**Пример: обработка ошибок в представлении**:

Создадим представление для создания пользователя, где будет производиться проверка данных и возвращаться подробное сообщение в случае ошибки.

```plaintext
my_project/
└── my_app/
    ├── views.py  # файл, содержащий представление
    ├── models.py
    └── serializers.py
```

В файле `views.py` создаём представление `UserCreateView`, где будет обрабатываться ошибка валидации:

```python
# views.py
from rest_framework import status
from rest_framework.response import Response
from rest_framework.views import APIView
from .serializers import UserSerializer

class UserCreateView(APIView):
    def post(self, request, *args, **kwargs):
        serializer = UserSerializer(data=request.data)
        
        # Проверка данных на валидность
        if not serializer.is_valid():
            return Response(
                {
                    'error': 'Validation Error',
                    'status': status.HTTP_400_BAD_REQUEST,
                    'details': serializer.errors  # детализированная информация о поле
                },
                status=status.HTTP_400_BAD_REQUEST
            )
        
        # Если данные валидны, сохраняем их
        serializer.save()
        return Response(serializer.data, status=status.HTTP_201_CREATED)
```

**Пример JSON-ответа** при неверных данных:

```json
{
    "error": "Validation Error",
    "status": 400,
    "details": {
        "username": ["This field is required."]
    }
}
```

### 7.5 Типовые примеры ошибок и JSON-ответов для них

1. **Ошибка 400 Bad Request**:
   ```json
   {
       "error": "Invalid request",
       "status": 400,
       "details": {
           "username": ["This field is required."]
       }
   }
   ```

2. **Ошибка 401 Unauthorized**:
   ```json
   {
       "error": "Unauthorized",
       "status": 401,
       "details": "Authentication credentials were not provided."
   }
   ```

3. **Ошибка 403 Forbidden**:
   ```json
   {
       "error": "Forbidden",
       "status": 403,
       "details": "You do not have permission to perform this action."
   }
   ```

4. **Ошибка 404 Not Found**:
   ```json
   {
       "error": "Not Found",
       "status": 404,
       "details": "The requested resource does not exist."
   }
   ```

5. **Ошибка 500 Internal Server Error**:
   ```json
   {
       "error": "Internal Server Error",
       "status": 500,
       "details": "An unexpected error occurred on the server."
   }
   ```


### **Настройка валидации на уровне модели**

DRF также позволяет настраивать валидацию на уровне модели, чтобы при сохранении данных в базе данных автоматически возвращались детализированные сообщения об ошибках.

**Пример: валидация модели**:

Создаём модель `Product` в `models.py` и добавляем метод `clean` для проверки валидности данных:

```python
# models.py
from django.db import models
from django.core.exceptions import ValidationError

class Product(models.Model):
    name = models.CharField(max_length=100)
    price = models.DecimalField(max_digits=10, decimal_places=2)

    def clean(self):
        if self.price <= 0:
            raise ValidationError("Price must be greater than zero.")
```

**Обработка ошибки в представлении**:

Создаём представление `ProductCreateView` в `views.py`, где проверяем и сохраняем данные:

```python
# views.py
from rest_framework import status
from rest_framework.views import APIView
from rest_framework.response import Response
from .models import Product

class ProductCreateView(APIView):
    def post(self, request, *args, **kwargs):
        try:
            product = Product(**request.data)
            product.full_clean()  # запуск валидации модели
            product.save()
            return Response({'message': 'Product created'}, status=status.HTTP_201_CREATED)
        except ValidationError as e:
            return Response(
                {
                    'error': 'Validation Error',
                    'status': status.HTTP_400_BAD_REQUEST,
                    'details': e.message_dict  # вывод ошибок валидации
                },
                status=status.HTTP_400_BAD_REQUEST
            )
```

**

Ответ при ошибке**:

```json
{
    "error": "Validation Error",
    "status": 400,
    "details": {
        "price": ["Price must be greater than zero."]
    }
}
```